<a href="https://colab.research.google.com/github/ASIKoba/dw_matrix_road_sign/blob/master/day5tr3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'


/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
#dzisiaj jest tematem dobór parametrów


In [32]:
import pandas as pd
import numpy as np
import os
import datetime


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score 
%load_ext tensorboard
from hyperopt import hp,STATUS_OK, tpe, Trials, fmin 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
ls data

signnames.csv  test.p  train.p


In [0]:
train =pd.read_pickle('data/train.p')
test =pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
if y_train.ndim==1: y_train=to_categorical(y_train) 
if y_test.ndim==1: y_test=to_categorical(y_test) 

In [0]:
input_shape = X_train.shape[1:]
num_classes =y_train.shape[1]

In [0]:
def train_model(get_model, X_train, y_train, params_fit={}):
    model.compile(loss='categorical_crossentropy', optimizer ='Adam', metrics=['accuracy'])

    logdir =os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback =tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


    model.fit(
        X_train, 
        y_train,
        batch_size=params_fit.get('batch_size',128),  #dzielimy na paczki , które służa do uczenia się 
        epochs=params_fit.get('epochs',5), # epoki, liczba iteracji przezktóre przepuszczamy wszystkie dane
        verbose=params_fit.get('verbose',1), #wypisujemy jaki mamy stan
        validation_data=params_fit.get('validation_data',(X_train,y_train)), #obserwowanie na ile model sie wyuczył
        callbacks=[tensorboard_callback]
        )
    return model


In [0]:
def predict(model_trained,X_test,y_test, scoring =accuracy_score ):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)


In [0]:
def train_and_predict(model):
  model_trained = train_model(model,X_train,y_train)
  return predict(model_trained,X_test, y_test)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32,kernel_size=(3,3),activation='relu', input_shape = input_shape),
                    Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding ='same'),
                    MaxPool2D(),
                    Dropout(0.3), 

                    Conv2D(filters=64,kernel_size=(3,3),activation='relu', padding ='same'),
                    Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                    
                    Conv2D(filters=64,kernel_size=(3,3),activation='relu', padding ='same'),
                    Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Flatten(),
                     
                    Dense(1024, activation='relu'), 
                    Dropout(0.3), 
                     
                    Dense(1024, activation='relu'), 
                    Dropout(0.3), 

                    Dense(num_classes, activation='softmax'),
                    ])
  

#train_and_predict(get_cnn_v4(input_shape, num_classes) )  

In [0]:
X_train_gray =color.rgb2gray(X_train).reshape(-1,32,32,1)
X_test_gray =color.rgb2gray(X_test).reshape(-1,32,32,1)

In [21]:
model = get_cnn_v5((32,32,1), num_classes)
params_fit={
    'epochs':20
}
model_trained = train_model(model, X_train_gray, y_train, params_fit)
predict(model_trained, X_test_gray, y_test)

Epoch 1/20
272/272 [==============================] - 5s 19ms/step - loss: 2.4476 - accuracy: 0.3007 - val_loss: 0.8264 - val_accuracy: 0.7374
Epoch 2/20
272/272 [==============================] - 5s 17ms/step - loss: 0.5687 - accuracy: 0.8156 - val_loss: 0.1339 - val_accuracy: 0.9574
Epoch 3/20
272/272 [==============================] - 5s 17ms/step - loss: 0.2465 - accuracy: 0.9205 - val_loss: 0.0644 - val_accuracy: 0.9816
Epoch 4/20
272/272 [==============================] - 5s 17ms/step - loss: 0.1574 - accuracy: 0.9500 - val_loss: 0.0354 - val_accuracy: 0.9905
Epoch 5/20
272/272 [==============================] - 5s 17ms/step - loss: 0.1205 - accuracy: 0.9605 - val_loss: 0.0300 - val_accuracy: 0.9915
Epoch 6/20
272/272 [==============================] - 5s 17ms/step - loss: 0.0950 - accuracy: 0.9714 - val_loss: 0.0154 - val_accuracy: 0.9958
Epoch 7/20
272/272 [==============================] - 5s 17ms/step - loss: 0.0848 - accuracy: 0.9728 - val_loss: 0.0216 - val_accuracy: 0.9928

0.9786848072562359

In [37]:
model_trained.evaluate(X_test,y_test)[1]

138/138 [==============================] - 0s 4ms/step - loss: 0.2076 - accuracy: 0.9669


0.9668934345245361

In [0]:
def get_model(params):
  return Sequential([
                    Conv2D(filters=32,kernel_size=(3,3),activation='relu', input_shape = input_shape),
                    Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding ='same'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_one']), 

                    Conv2D(filters=64,kernel_size=(3,3),activation='relu', padding ='same'),
                    Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_two']),
                    
                    Conv2D(filters=128,kernel_size=(3,3),activation='relu', padding ='same'),
                    Conv2D(filters=128,kernel_size=(3,3),activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_three']),
                     
                    Flatten(),
                     
                    Dense(1024, activation='relu'), 
                    Dropout(params['dropout_dense_block_one']),
                     
                    Dense(1024, activation='relu'), 
                    Dropout(params['dropout_dense_block_two']),

                    Dense(num_classes, activation='softmax'),
                    ])
  


In [0]:
def func_obj(params):
  #model = get_cnn_v5(input_shape, num_classes)
  model=get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics = ['accuracy'])

  model.fit(
        X_train, 
        y_train,
        batch_size=int(params.get('batch_size',128)),  #dzielimy na paczki , które służa do uczenia się 
        epochs=5, # epoki, liczba iteracji przezktóre przepuszczamy wszystkie dane, było 7
        verbose=0
        )
  score = model.evaluate(X_test,y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss':-accuracy, 'status':STATUS_OK, 'model':model}


In [25]:
model = get_cnn_v5(input_shape, num_classes)

model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 19ms/step - loss: 2.5263 - accuracy: 0.3088 - val_loss: 1.1321 - val_accuracy: 0.6742
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.8722 - accuracy: 0.7243 - val_loss: 0.2481 - val_accuracy: 0.9225
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.4115 - accuracy: 0.8744 - val_loss: 0.1160 - val_accuracy: 0.9656
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2736 - accuracy: 0.9191 - val_loss: 0.0486 - val_accuracy: 0.9864
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2026 - accuracy: 0.9386 - val_loss: 0.0385 - val_accuracy: 0.9904


0.9668934240362812

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 50,200,20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one',0,1),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',0,1),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',0,1),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one',0,1),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two',0,1),
}

In [54]:
best= fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 140.0, 'dropout_cnn_block_one': 0.34531484478777247, 'dropout_cnn_block_three': 0.4339459276118087, 'dropout_cnn_block_two': 0.3378860485090417, 'dropout_dense_block_one': 0.6368698324280424, 'dropout_dense_block_two': 0.4188297827654853}
accuracy=0.9585034251213074
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.3082467150023696, 'dropout_cnn_block_three': 0.4334038659638466, 'dropout_cnn_block_two': 0.41643226489152835, 'dropout_dense_block_one': 0.4266684326604927, 'dropout_dense_block_two': 0.6191700785218219}
accuracy=0.9278911352157593
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.38876607587456175, 'dropout_cnn_block_three': 0.3908949348649574, 'dropout_cnn_block_two': 0.4644169242218541, 'dropout_dense_block_one': 0.33695847266550943, 'dropout_dense_block_two': 0.3785342262446913}
accuracy=0.8981859683990479
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.44885169605869524, 'dropout_cnn_block_three': 0.3878630433025184, 'dropout_cnn_block_two': 0.32683966866

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100,200,10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one',0.3,0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',0.3,0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',0.3,0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one',0.3,0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two',0.3,0.7),
}

In [51]:
best= fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 100.0, 'dropout_cnn_block_one': 0.5, 'dropout_cnn_block_three': 0.5, 'dropout_cnn_block_two': 0.5, 'dropout_dense_block_one': 0.5, 'dropout_dense_block_two': 0.5}
accuracy=0.8698412775993347
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.5, 'dropout_cnn_block_three': 0.5, 'dropout_cnn_block_two': 0.5, 'dropout_dense_block_one': 0.5, 'dropout_dense_block_two': 0.5}
accuracy=0.8761904835700989
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.5, 'dropout_cnn_block_three': 0.5, 'dropout_cnn_block_two': 0.5, 'dropout_dense_block_one': 0.5, 'dropout_dense_block_two': 0.5}
accuracy=0.8285714387893677
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.5, 'dropout_cnn_block_three': 0.5, 'dropout_cnn_block_two': 0.5, 'dropout_dense_block_one': 0.5, 'dropout_dense_block_two': 0.5}
accuracy=0.6517006754875183
{'batch_size': 60.0, 'dropout_cnn_block_one': 0.5, 'dropout_cnn_block_three': 0.5, 'dropout_cnn_block_two': 0.5, 'dropout_dense_block_one': 0.5, 'dropout_dense_block_two': 0.5}